In [1]:
import google.generativeai as genai

/home/gcasado0/proyectos/chatbot/chatbot/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
# Used to securely store your API key
import os

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

In [3]:
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-1.5-flash')

In [4]:
context='Eres un chabot'
query='¿Cómo te llamas?'
response = model.generate_content(f"Contexto: {context}\n\nPregunta: {query}")

In [6]:
response.text

'No tengo nombre.  Soy un modelo de lenguaje grande.\n'

In [7]:
text = f"""
You should express what you want a model to do by  
providing instructions that are as clear and 
specific as you can possibly make them. 
This will guide the model towards the desired output, 
and reduce the chances of receiving irrelevant 
or incorrect responses. Don't confuse writing a 
clear prompt with writing a short prompt. 
In many cases, longer prompts provide more clarity 
and context for the model, which can lead to 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks 
into a single sentence.
```{text}```
"""
response = model.generate_content(prompt)
print(response.text)

To get the best results from a model, provide clear and specific instructions, even if that means using longer prompts to give sufficient context.



In [8]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

In [10]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = model.generate_content(prompt)
print(response.text)

The sentiment of the review is overwhelmingly **positive**.  The reviewer praises the lamp's features (storage, price), the fast shipping, the company's excellent customer service in resolving the broken string and missing part issues, and expresses overall satisfaction with both the product and the company.



In [11]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = model.generate_content(prompt)
print(response.text)

satisfaction, gratitude, relief, happiness,  trust



In [12]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""

response = model.generate_content(prompt)
print(response.text)

```json
{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp",
  "Brand": "Lumina"
}
```



In [13]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = model.generate_content(prompt)
print(response.text)

Employee satisfaction, NASA, Government survey, Job satisfaction, Social Security



In [23]:
consulta = """
"Buenas tardes, 
al intentar iniciar el tramite de renovación de habilitación Municipal sin modificaciones el sistema nos devuelve el siguiente mensaje:
""Advertencia
Deberás realizar el Trámite de Renovación con modificaciones - Modificar Actividad porque hemos detectado diferencias entre la/s actividad/es y el/los destino/s del último certificado otorgado.""
como debemos proceder para renovar la habilitación y cuales son las diferencias detectadas?
Gracias
Saludos
"""

prompt = f"""
Determine cinco temas que se están tratando en el siguiente texto, que está delimitado por tres comillas invertidas.
Haga que cada elemento tenga una o dos palabras.
Formatee su respuesta como una lista de elementos separados por comas.
Indique el sentimiento general del texto.
Indique si el remitente está expresando enojo.
Texto: '''{consulta}'''
Salida en formato JSON:
"Temas": 
"Sentimiento": 
"Enojo": "Si/No"
"""
response = model.generate_content(prompt)
print(response.text)

```json
{
  "Temas": "Renovación, Habilitación, Municipal, Sistema, Diferencias",
  "Sentimiento": "Frustración",
  "Enojo": "No"
}
```



Recorrer la lista de consultas de habilitaciones y obtener tematicas claves y sentimientos de las mismas


In [25]:
import pandas as pd

data = pd.read_excel('/home/gcasado0/proyectos/chatbot/Consultas_habilitaciones.xlsx')
data.head()

,Serial number,Submission ID,Submission URI,Creado,La consulta es,Domicilio: Dirección del consultante,Domicilio: ubi_o_outRos,Domicilio: Piso,Domicilio: Dpto,Domicilio: id_calle,...,Número de: establecimiento o certficado de hablitación o TGI,Contenido consulta,codigo_gestion,PDF,cuit,nivel,mail,access_token,cod_tramite,Análisis de la rta
0,499,1794755,/node/1754?check_logged_in=1,2024-01-03 09:26:32,Para una tercera persona,NUEVE DE JULIO y RODRIGUEZ MARTIN,NaN,NaN,NaN,72000.0,...,3185320,- la actividad declarada 829100 corresponde a ...,3423-2024,https://gestorpic.rosario.gob.ar/inicio/webfor...,27147589943,,alepedrido@hotmail.com,eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIi...,755,Lo enviaron al CMD dado que con el nro. de est...
1,500,1798854,/node/1754,2024-01-04 09:13:08,Para mí,CORDOBA 1000,NaN,NaN,NaN,43000.0,...,NaN,"Buenos días. Quisiera consultar ""Qué tipo de h...",5585-2024,https://gestorpic.rosario.gob.ar/inicio/webfor...,27373328451,,jumadacunhakessler@gmail.com,eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIi...,755,NaN
2,501,1800249,/node/1754?check_logged_in=1,2024-01-04 13:40:34,Para mí,SARMIENTO DOMINGO FAUSTINO 1242,NaN,1242,NaN,87300.0,...,NaN,quiero saber como se realiza una habilitacion ...,[webform:handler:remote_post_pic:completed:dat...,https://gestorpic.rosario.gob.ar/inicio/webfor...,27265509180,3,sanysilva@icloud.com,eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIi...,755,NaN
3,502,1804219,/node/1754,2024-01-05 15:00:21,Para mí,SAN JUAN 3011,NaN,NaN,NaN,85650.0,...,NaN,Hola quiero consultar ya que quiero realizar u...,8536-2024,https://gestorpic.rosario.gob.ar/inicio/webfor...,27426087443,2,sianuu.robledo@gmail.com,eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIi...,755,NaN
4,503,1809440,/node/1754?check_logged_in=1,2024-01-08 08:57:03,Para mí,NECOCHEA GRAL. MARIANO 3453,NaN,NaN,NaN,71000.0,...,NaN,Para hacerme el monotributo me pide el contado...,11972-2024,https://gestorpic.rosario.gob.ar/inicio/webfor...,20302544124,2,chukycuervosl@hotmail.com,eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIi...,755,NaN


In [31]:
consulta = data['Contenido consulta'][0]

In [32]:
prompt = f"""
Determine cinco temas que se están tratando en el siguiente texto, que está delimitado por tres comillas invertidas.
Haga que cada elemento tenga una o dos palabras.
Formatee su respuesta como una lista de elementos separados por comas.
Indique el sentimiento general del texto.
Indique si el remitente está expresando enojo.
Texto: '''{consulta}'''
Salida en formato JSON:
"Temas": 
"Sentimiento": 
"Enojo": "Si/No"
"""
response = model.generate_content(prompt)
print(response.text)

```json
{
  "Temas": "Código actividad, Alicuotas, Clasificación, Plus pago,  Polirrubro",
  "Sentimiento": "Preocupación, confusión",
  "Enojo": "No"
}
```

